# AULA 4

A partir dessa aula nosso estudo será mais direcionado para o viés funcional de Scala. Nesta aula será apresentado o conceito e aplicação de *Pattern Matching*.

## Pattern Matching

Em várias linguagens existe um operador chamado *switch*, que funciona como uma sucessiva aplicação de *if* e *else*. Em Scala, o operador que faz isso é chamado de *match*. O exemplo abaixo demonstra um simples funcionamento do *match*, onde reescrevemos o encadeamento de estruturas de controle utilizando *pattern matching*:

In [1]:
val x = 5

if(x==5) println("x = 5") //caso x seja 5
else if(x==10) println("x = 10") //caso x seja 10
else println("x não é 5 nem 10") //caso x não seja 5 nem 10

x match {
    case 5 => println("x = 5") //caso x seja 5 
    case 10 => println("x = 10")// caso x seja 10
    case _ => println("x não é 5 nem 10") //caso x não seja nem 5 nem 10
}

x = 5
x = 5


x: Int = 5

Diferente de outras linguagens, o *pattern matching* aceita não só comparação com números inteiros, mas sim com qualquer valor:

In [2]:
val s = "olá"

s match {
    case "olá" => println("olá!")
    case "oi" => println("oi!")
    case _ => println("??")
}

olá!


s: String = "olá"

## Exemplo de aplicação: Expressões Lógicas

Vamos implementar expressões lógicas utilizando *pattern matching*. Primeiramente, vamos obter as classes bases para os operadores:

In [3]:
abstract class ExpressaoLogica {
    
    val tipo:String //adicionamos esse atributo para sabermos o nome da operação em questão
    
    //métodos para obter os valores das variáveis da expressão:
    
    def v1: ExpressaoLogica
    def v2: ExpressaoLogica
}

defined class ExpressaoLogica

In [4]:
class Constante(p:Boolean) extends ExpressaoLogica{
    
    val tipo: String = "Constante"
    def apply(): Boolean = p
    
    def v1: ExpressaoLogica = null
    def v2: ExpressaoLogica = null
}

defined class Constante

In [5]:
abstract class OperacaoUnaria(p: ExpressaoLogica) extends ExpressaoLogica{
    
    def v1: ExpressaoLogica = p
    def v2: ExpressaoLogica = null
}

defined class OperacaoUnaria

In [6]:
abstract class OperacaoBinaria(p: ExpressaoLogica, q: ExpressaoLogica) extends ExpressaoLogica{
    
    def v1: ExpressaoLogica = p
    def v2: ExpressaoLogica = q
}

defined class OperacaoBinaria

Vamos trabalhar utilizando os seguintes operadores lógicos:

* Ou (binário)
* E (binário)
* Não (unário)
* Implica (binário)

In [7]:
class Nao(p: ExpressaoLogica) extends OperacaoUnaria(p) {
    val tipo:String = "Não"
}

class Ou(p: ExpressaoLogica, q:ExpressaoLogica) extends OperacaoBinaria(p,q){
    val tipo: String = "Ou"
}

class E(p: ExpressaoLogica, q: ExpressaoLogica) extends OperacaoBinaria(p,q){
    val tipo: String = "E"
}

class Implica(p: ExpressaoLogica,q:ExpressaoLogica) extends OperacaoBinaria(p,q){
    val tipo: String = "Implica"
}

defined class Nao
defined class Ou
defined class E
defined class Implica

Vamos criar um object para receber uma ExpressaoLogica e resolvê-la recursivamente usando *pattern matching*

In [8]:
object Solver{
    def apply(expr: ExpressaoLogica):Boolean = expr.tipo match {
        
        case "Constante" => (expr.asInstanceOf[Constante])()
        
        case "Nao" => !(apply(expr.v1))
        case "Ou" => apply(expr.v1) || apply(expr.v2)
        case "E" => apply(expr.v1) && apply(expr.v2)
        case "Implica" => !(apply(expr.v1)) || apply(expr.v2)
        case _ => {
            println("Operação não reconhecida")
            false
        }
    }
}

val e = new Ou(
    new E(
        new Constante(true),new Constante(true)
    ),
    new Implica(
        new Constante(false),
        new Nao(
            new Constante(true)
        )
    )
)

Solver(e)

defined object Solver
e: Ou = $sess.cmd6Wrapper$Helper$Ou@7251d928
res7_2: Boolean = true

## Refinando o Pattern Matching: Case Class

Como vimos, podemos aplicar o operador *match* sobre qualquer tipo. Existe um recurso em Scala chamado de *case class*: uma classe que pode ser usada no *pattern matching* para obter os **valores utilizados para a criação do objeto**. Vamos ao exemplo a seguir:

In [9]:
abstract class Valor()

case class UmValor(a:Int) extends Valor

case class DoisValores(a: Int, b: String) extends Valor

case class TresValores(a: Int, b: Int, c:Int) extends Valor

val m: Valor = new DoisValores(5,"abacaxi")

m match {
    case UmValor(x) => print(s"Apenas um valor: $x")
    case DoisValores(x,y) => print(s"Dois valores: $x,$y")
    case TresValores(x,y,z) => print(s"Três valores: $x,$y,$z")
}

Dois valores: 5,abacaxi

defined class Valor
defined class UmValor
defined class DoisValores
defined class TresValores
m: Valor = DoisValores(5,abacaxi)

### Refinando o exemplo das Expressões Lógicas

Vamos transformar as classes anteriores em *case class* para que não precisemos de métodos para acessar os atributos. O código final ficará assim:

In [10]:
abstract class ExpressaoLogica

case class Constante(p: Boolean) extends ExpressaoLogica

case class Nao(p: ExpressaoLogica) extends ExpressaoLogica

case class Ou(p: ExpressaoLogica, q: ExpressaoLogica) extends ExpressaoLogica

case class E(p: ExpressaoLogica, q: ExpressaoLogica) extends ExpressaoLogica

case class Implica(p: ExpressaoLogica, q: ExpressaoLogica) extends ExpressaoLogica

object Solver {
    
    def apply(expr: ExpressaoLogica): Boolean = expr match {
        case Constante(p) => p
        case Nao(p) => !(apply(p))
        case Ou(p,q) => apply(p) || apply(q)
        case E(p,q)  => apply(p) && apply(q)
        case Implica(p,q) => !(apply(p)) || apply(q)
        case _  => {
            println("Operação não reconhecida")
            false
        }
    }
}

defined class ExpressaoLogica
defined class Constante
defined class Nao
defined class Ou
defined class E
defined class Implica
defined object Solver

In [11]:
val e = new Ou(
    new E(
        new Constante(true),new Constante(true)
    ),
    new Implica(
        new Constante(false),
        new Nao(
            new Constante(true)
        )
    )
)

Solver(e)

e: Ou = Ou(E(Constante(true),Constante(true)),Implica(Constante(false),Nao(Constante(true))))
res10_1: Boolean = true

 ## Exercícios
 
 ### Escreva implementações para Expressões Numéricas como soma, subtração, divisão, etc., utilizando *pattern matching* e *case class*, conforme o exemplo das Expressões Lógicas.

In [ ]:
abstract class Expressao

In [96]:
case class Somar(a:Double,b:Double) extends Expressao
case class Subtrair(a:Double,b:Double) extends Expressao
case class Multiplicar(a:Double,b:Double) extends Expressao
case class Dividir(a:Double,b:Double) extends Expressao

defined class Somar
defined class Subtrair
defined class Multiplicar
defined class Dividir

In [97]:
object Solução{ 
    
    def apply(expr: Expressao): Unit = expr match {
        case Somar(a,b) => println("Soma: "+(a+b))
        case Subtrair(a,b)=> println("Subtração: "+(a-b))
        case Multiplicar(a,b) => println("Multiplicação: "+(a*b))
        case Dividir(a,b) => println("Divisão: "+(a/b))
    }
}

defined object Solução

In [111]:
val soma = new Somar(5.5,10)

soma: Somar = Somar(5.5, 10.0)

In [112]:
Solução(soma)

Soma: 15.5


In [110]:
val diferença = new Subtrair(10.5,5)
Solução(diferença)

Subtração: 5.5


diferença: Subtrair = Subtrair(10.5, 5.0)

In [103]:
val produto = new Multiplicar(-2,-8.5)
Solução(produto)

Multiplicação: 17.0


produto: Multiplicar = Multiplicar(-2.0, -8.5)

In [102]:
val divisão = new Dividir(10,4)
Solução(divisão)

Divisão: 2.5


divisão: Dividir = Dividir(10.0, 4.0)